In [1]:
import pandas as pd
import re

cc_who = pd.read_csv('data/cc_who.csv')
cc_noc = pd.read_csv('data/cc_noc.csv')

In [2]:
print(cc_who.shape, cc_noc.shape)
print(cc_who.columns, cc_noc.columns)

(185, 2) (209, 2)
Index(['Country Code', 'Country Name'], dtype='object') Index(['NOC', 'Team'], dtype='object')


In [3]:
from data.used_cc_who import used_cc_who
len(used_cc_who)

183

In [4]:
# Filter missing lines
print(used_cc_who - set(cc_who['Country Name']))
cc_who = cc_who[cc_who['Country Name'].isin(used_cc_who & set(cc_who['Country Name']))]
cc_who.shape

{'Bolivia Plurinational States of '}


(182, 2)

In [5]:
def join_by_cc(who, noc):
    res = {}
    
    
    codes_intersect = set(who.keys()) & set(noc.keys())
    
    for code in codes_intersect:
        k_noc = noc[code]
        k_who = who[code]
        k = k_noc if len(k_noc) < len(k_who) else k_who
        
        res[k] = {'who':code, 'noc':code}
    
    return res    

In [6]:
def join_by_name(who, noc, clusters):
    for code_who, name_who in who.items():
        for code_noc, name_noc in noc.items():
            if name_who == name_noc and code_who != code_noc:
                clusters[min(name_who, name_noc)] = {'noc': code_noc, 'who': code_who}
    
    return clusters

In [7]:
who = {b: re.sub(r'\([^)]*\)', '', a) for a, b in zip(cc_who['Country Name'], cc_who['Country Code'])}
noc = {b: re.sub(r'\([^)]*\)', '', a) for a, b in zip(cc_noc['Team'], cc_noc['NOC'])}


countries = join_by_cc(who, noc)
countries = join_by_name(who, noc, countries)

countries = pd.DataFrame(data=[(k, v['who'], v['noc']) for k, v in countries.items()], columns=['country', 'code_who', 'code_noc'])
countries

,country,code_who,code_noc
0,Ukraine,UKR,UKR
1,Jordan,JOR,JOR
2,Macedonia,MKD,MKD
3,Slovakia,SVK,SVK
4,Venezuela,VEN,VEN
...,...,...,...
171,Vanuatu,VUT,VAN
172,Samoa,WSM,SAM
173,South Africa,ZAF,RSA
174,Zambia,ZMB,ZAM


In [10]:
countries.to_csv('data/country_codes.csv', index=False)